### <font color='blue'>Import all packages</font> ###

In [80]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

In [82]:

import tensorflow as tf
print(tf.__version__)

2.7.0


In [83]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

### <font color='blue'> Preprocessing and cleaning functions </font> ###

In [84]:
!cp "/content/drive/My Drive/summarized_sc.txt" "./summarized_sc.txt"
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"
!cp "/content/drive/My Drive/sc_model_0.txt" "./sc_model_0.txt"
!cp "/content/drive/My Drive/sc_model_1.txt" "./sc_model_1.txt"
!cp "/content/drive/My Drive/sc_model_2.txt" "./sc_model_2.txt"
!cp "/content/drive/My Drive/sc_model_3.txt" "./sc_model_3.txt"
!cp "/content/drive/My Drive/sc_model_4.txt" "./sc_model_4.txt"
!cp "/content/drive/My Drive/sc_model_5.txt" "./sc_model_5.txt"

### <font color='blue'> Reading and Cleaning the Dataset </font> ###

In [85]:
#@title
# data_file='./data/spam.csv'
# data=pd.read_csv(data_file,encoding='ISO-8859-1')

# data.head()



#############
# test_set = pd.read_csv("toxic_span_practice.csv")
# #test_set=test_set[test_set['spans'].str.len() != 0]
# train_final_set=[]
# train_set = pd.read_csv("toxic_span_train.csv")

# train_final_set=train_set

# train_set=train_set[train_set['spans'].str.len() != 0]
# trial_set = pd.read_csv("toxic_span_trial.csv")

# train_final_set=train_final_set.append(trial_set,ignore_index=True)

# trial_set=trial_set[trial_set['spans'].str.len() != 0]
# toxic_set=train_set.append(trial_set,ignore_index=True)
# toxic_set['label']='TOXIC'
# nontoxic_set = pd.read_csv("nontoxic_span_train.csv")
# classify_data_1=toxic_set[['text']]
# classify_data_1['label']='TOXIC'
# classify_data_2=nontoxic_set[['text']]
# classify_data_2['label']='NON_TOXIC'

# classify_data=classify_data_1.append(classify_data_2,ignore_index=True)
# classify_data=shuffle(classify_data)
# print(toxic_set)
# print(nontoxic_set)
# print(classify_data)
# print(test_set)
# print(train_final_set)
##############



# toxic_span_dataset=test_set.append(train_set,ignore_index=True)
# toxic_span_dataset = toxic_span_dataset.append(trial_set,ignore_index=True)
# classify_data=nontoxic.append(train_set)

In [86]:

fh = open('sc_model_0.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)


8419
8419
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

#### <font color='purple'> Removing Unnamed Columns, dropping NaN data and resetting the index after dropping some rows/columns containing NaN dataset and finally shuffling the dataset </font> #### 

#### <font color='purple'> Rename v1:label and v2:text, converting 'ham' labels to '0' and 'spam' to '1', saving it to the 'gt' (ground truth) column and applying the preprocess function to the dataset </font> #### 

In [87]:
gc.collect()
tf.keras.backend.clear_session()

### <font color='blue'> Loading DistilBERT Tokenizer and the DistilBERT model </font> ###

In [88]:
#dbert_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


### <font color='blue'> Preparing input for the model </font> ###

In [89]:
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)

(8419, 8419, 8419)

### <font color='blue'> Create a basic NN model using DistilBERT embeddings to get the predictions </font> ###

In [90]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

<blockquote> <font color='green'> Feel free to add more Dense and Dropout layers with variable units and the regularizers </font> </blockquote>

In [91]:
model_0=create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

In [92]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  8 ... 10 11  9]
[   1    2    3 ... 8417 8418 8419]


In [93]:
len(input_ids),len(attention_masks),len(labels), len(filenumbers)

(8419, 8419, 8419, 8419)

In [94]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)
#train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)

#print(train_fnum)

In [95]:
# print(type(train_inp))
# print(len(train_fnum))
# print(len(val_fnum))
new_train_inp_0=[]
new_val_inp_0=[]
new_train_label_0=[]
new_val_label_0=[]
new_train_mask_0=[]
new_train_fnum_0=[]
new_val_fnum_0=[]
new_val_mask_0=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_0.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_0.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_0.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_0.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_0.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_0.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_0.append(val_mask[i])
    new_val_fnum_0.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_0=np.array(new_train_inp_0)
new_val_inp_0=np.array(new_val_inp_0)
new_train_label_0=np.array(new_train_label_0)
new_val_label_0=np.array(new_val_label_0)
new_train_mask_0=np.array(new_train_mask_0)
new_train_fnum_0=np.array(new_train_fnum_0)
new_val_fnum_0=np.array(new_val_fnum_0)
new_val_mask_0=np.array(new_val_mask_0)

In [96]:

# print(val_inp)
# print(val_mask)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_0.shape,new_val_inp_0.shape,new_train_label_0.shape,new_val_label_0.shape,new_train_mask_0.shape,new_val_mask_0.shape,new_train_fnum_0.shape,new_val_fnum_0.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_0.compile(loss=loss,optimizer=optimizer, metrics=[metric])

Train inp shape (7577, 512) Val input shape (842, 512)
Train label shape (7577,) Val label shape (842,)
Train attention mask shape (7577, 512) Val attention mask shape (842, 512)
Train filenumber shape (7577,) Val filenumber shape (842,)


In [97]:
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 11 06:31:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    40W / 300W |  15237MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [98]:
history=model_0.fit([new_train_inp_0,new_train_mask_0],new_train_label_0,batch_size=8,epochs=1,validation_data=([new_val_inp_0,new_val_mask_0],new_val_label_0),callbacks=callbacks)
#history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=1,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - 170s 171ms/step - loss: 6.8978 - accuracy: 0.6192 - val_loss: 6.0207 - val_accuracy: 0.6781


In [99]:

fh = open('sc_model_1.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

new_label_list=[]
#summarized_data['label'] = label_list

#print(summarized_data)
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

num_classes=len(summarized_data.label.unique())
summarized_data.head()
print(num_classes)


gc.collect()
tf.keras.backend.clear_session()

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)


def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

model_1=create_model()

input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

len(input_ids),len(attention_masks),len(labels), len(filenumbers)

print(type(train_inp))
print(len(train_fnum))
print(len(val_fnum))
new_train_inp_1=[]
new_val_inp_1=[]
new_train_label_1=[]
new_val_label_1=[]
new_train_mask_1=[]
new_train_fnum_1=[]
new_val_fnum_1=[]
new_val_mask_1=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_1.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_1.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_1.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_1.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_1.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_1.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_1.append(val_mask[i])
    new_val_fnum_1.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_1=np.array(new_train_inp_1)
new_val_inp_1=np.array(new_val_inp_1)
new_train_label_1=np.array(new_train_label_1)
new_val_label_1=np.array(new_val_label_1)
new_train_mask_1=np.array(new_train_mask_1)
new_train_fnum_1=np.array(new_train_fnum_1)
new_val_fnum_1=np.array(new_val_fnum_1)
new_val_mask_1=np.array(new_val_mask_1)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_1.shape,new_val_inp_1.shape,new_train_label_1.shape,new_val_label_1.shape,new_train_mask_1.shape,new_val_mask_1.shape,new_train_fnum_1.shape,new_val_fnum_1.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_1.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model_1.fit([new_train_inp_1,new_train_mask_1],new_train_label_1,batch_size=8,epochs=1,validation_data=([new_val_inp_1,new_val_mask_1],new_val_label_1),callbacks=callbacks)

7332
7332
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  8 ... 10 11  9]
[   1    2    3 ... 8417 8418 8419]
<class 'numpy.ndarray'>
7577
842
Train inp shape (6601, 512) Val input shape (731, 512)
Train label shape (6601,) Val label shape (731,)
Train attention mask shape (6601, 512) Val attention mask shape (731, 512)
Train filenumber shape (6601,) Val filenumber shape (731,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


826/826 [==============================] - 149s 172ms/step - loss: 6.9350 - accuracy: 0.6226 - val_loss: 6.0196 - val_accuracy: 0.7086


In [100]:

fh = open('sc_model_2.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

new_label_list=[]
#summarized_data['label'] = label_list

#print(summarized_data)
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

num_classes=len(summarized_data.label.unique())
summarized_data.head()
print(num_classes)


gc.collect()
tf.keras.backend.clear_session()

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)


def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

model_2=create_model()

input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

len(input_ids),len(attention_masks),len(labels), len(filenumbers)

print(type(train_inp))
print(len(train_fnum))
print(len(val_fnum))
new_train_inp_2=[]
new_val_inp_2=[]
new_train_label_2=[]
new_val_label_2=[]
new_train_mask_2=[]
new_train_fnum_2=[]
new_val_fnum_2=[]
new_val_mask_2=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_2.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_2.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_2.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_2.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_2.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_2.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_2.append(val_mask[i])
    new_val_fnum_2.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_2=np.array(new_train_inp_2)
new_val_inp_2=np.array(new_val_inp_2)
new_train_label_2=np.array(new_train_label_2)
new_val_label_2=np.array(new_val_label_2)
new_train_mask_2=np.array(new_train_mask_2)
new_train_fnum_2=np.array(new_train_fnum_2)
new_val_fnum_2=np.array(new_val_fnum_2)
new_val_mask_2=np.array(new_val_mask_2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_2.shape,new_val_inp_2.shape,new_train_label_2.shape,new_val_label_2.shape,new_train_mask_2.shape,new_val_mask_2.shape,new_train_fnum_2.shape,new_val_fnum_2.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_2.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model_2.fit([new_train_inp_2,new_train_mask_2],new_train_label_2,batch_size=8,epochs=1,validation_data=([new_val_inp_2,new_val_mask_2],new_val_label_2),callbacks=callbacks)

6722
6722
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  8 ...  2  1 10]
[   1    2    3 ... 8413 8414 8417]
<class 'numpy.ndarray'>
7577
842
Train inp shape (6054, 512) Val input shape (668, 512)
Train label shape (6054,) Val label shape (668,)
Train attention mask shape (6054, 512) Val attention mask shape (668, 512)
Train filenumber shape (6054,) Val filenumber shape (668,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


757/757 [==============================] - 138s 172ms/step - loss: 7.0319 - accuracy: 0.6077 - val_loss: 6.1109 - val_accuracy: 0.7051


In [101]:

fh = open('sc_model_3.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

new_label_list=[]
#summarized_data['label'] = label_list

#print(summarized_data)
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

num_classes=len(summarized_data.label.unique())
summarized_data.head()
print(num_classes)


gc.collect()
tf.keras.backend.clear_session()

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)


def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

model_3=create_model()

input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

len(input_ids),len(attention_masks),len(labels), len(filenumbers)

print(type(train_inp))
print(len(train_fnum))
print(len(val_fnum))
new_train_inp_3=[]
new_val_inp_3=[]
new_train_label_3=[]
new_val_label_3=[]
new_train_mask_3=[]
new_train_fnum_3=[]
new_val_fnum_3=[]
new_val_mask_3=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_3.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_3.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_3.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_3.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_3.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_3.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_3.append(val_mask[i])
    new_val_fnum_3.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_3=np.array(new_train_inp_3)
new_val_inp_3=np.array(new_val_inp_3)
new_train_label_3=np.array(new_train_label_3)
new_val_label_3=np.array(new_val_label_3)
new_train_mask_3=np.array(new_train_mask_3)
new_train_fnum_3=np.array(new_train_fnum_3)
new_val_fnum_3=np.array(new_val_fnum_3)
new_val_mask_3=np.array(new_val_mask_3)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_3.shape,new_val_inp_3.shape,new_train_label_3.shape,new_val_label_3.shape,new_train_mask_3.shape,new_val_mask_3.shape,new_train_fnum_3.shape,new_val_fnum_3.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_3.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model_3.fit([new_train_inp_3,new_train_mask_3],new_train_label_3,batch_size=8,epochs=1,validation_data=([new_val_inp_3,new_val_mask_3],new_val_label_3),callbacks=callbacks)

6005
6005
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  8 ...  2  1 10]
[   1    2    3 ... 8413 8414 8417]
<class 'numpy.ndarray'>
7577
842
Train inp shape (5410, 512) Val input shape (595, 512)
Train label shape (5410,) Val label shape (595,)
Train attention mask shape (5410, 512) Val attention mask shape (595, 512)
Train filenumber shape (5410,) Val filenumber shape (595,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


677/677 [==============================] - 124s 172ms/step - loss: 7.1716 - accuracy: 0.6065 - val_loss: 6.3574 - val_accuracy: 0.7025


In [102]:

fh = open('sc_model_4.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

new_label_list=[]
#summarized_data['label'] = label_list

#print(summarized_data)
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

num_classes=len(summarized_data.label.unique())
summarized_data.head()
print(num_classes)


gc.collect()
tf.keras.backend.clear_session()

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)


def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

model_4=create_model()

input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

len(input_ids),len(attention_masks),len(labels), len(filenumbers)

print(type(train_inp))
print(len(train_fnum))
print(len(val_fnum))
new_train_inp_4=[]
new_val_inp_4=[]
new_train_label_4=[]
new_val_label_4=[]
new_train_mask_4=[]
new_train_fnum_4=[]
new_val_fnum_4=[]
new_val_mask_4=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_4.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_4.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_4.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_4.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_4.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_4.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_4.append(val_mask[i])
    new_val_fnum_4.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_4=np.array(new_train_inp_4)
new_val_inp_4=np.array(new_val_inp_4)
new_train_label_4=np.array(new_train_label_4)
new_val_label_4=np.array(new_val_label_4)
new_train_mask_4=np.array(new_train_mask_4)
new_train_fnum_4=np.array(new_train_fnum_4)
new_val_fnum_4=np.array(new_val_fnum_4)
new_val_mask_4=np.array(new_val_mask_4)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_4.shape,new_val_inp_4.shape,new_train_label_4.shape,new_val_label_4.shape,new_train_mask_4.shape,new_val_mask_4.shape,new_train_fnum_4.shape,new_val_fnum_4.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_4.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model_4.fit([new_train_inp_4,new_train_mask_4],new_train_label_4,batch_size=8,epochs=1,validation_data=([new_val_inp_4,new_val_mask_4],new_val_label_4),callbacks=callbacks)

5275
5275
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  2 ...  2  1 10]
[   1    2    4 ... 8413 8414 8417]
<class 'numpy.ndarray'>
7577
842
Train inp shape (4758, 512) Val input shape (517, 512)
Train label shape (4758,) Val label shape (517,)
Train attention mask shape (4758, 512) Val attention mask shape (517, 512)
Train filenumber shape (4758,) Val filenumber shape (517,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


595/595 [==============================] - 110s 172ms/step - loss: 7.2487 - accuracy: 0.5849 - val_loss: 6.4007 - val_accuracy: 0.6944


In [103]:

fh = open('sc_model_5.txt')
filenums=[]
records=[]
for record in fh:
    #print(record)
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    #print(filenum)

    #print(modelnum)
    records.append(text)
    #print(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

new_label_list=[]
#summarized_data['label'] = label_list

#print(summarized_data)
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

num_classes=len(summarized_data.label.unique())
summarized_data.head()
print(num_classes)


gc.collect()
tf.keras.backend.clear_session()

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels), len(filenumbers)


def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

model_5=create_model()

input_ids=[]
attention_masks=[]



for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])




input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

len(input_ids),len(attention_masks),len(labels), len(filenumbers)

print(type(train_inp))
print(len(train_fnum))
print(len(val_fnum))
new_train_inp_5=[]
new_val_inp_5=[]
new_train_label_5=[]
new_val_label_5=[]
new_train_mask_5=[]
new_train_fnum_5=[]
new_val_fnum_5=[]
new_val_mask_5=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_5.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_5.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_5.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_5.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_5.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_5.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_5.append(val_mask[i])
    new_val_fnum_5.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_5=np.array(new_train_inp_5)
new_val_inp_5=np.array(new_val_inp_5)
new_train_label_5=np.array(new_train_label_5)
new_val_label_5=np.array(new_val_label_5)
new_train_mask_5=np.array(new_train_mask_5)
new_train_fnum_5=np.array(new_train_fnum_5)
new_val_fnum_5=np.array(new_val_fnum_5)
new_val_mask_5=np.array(new_val_mask_5)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}\nTrain filenumber shape {} Val filenumber shape {}'.format(new_train_inp_5.shape,new_val_inp_5.shape,new_train_label_5.shape,new_val_label_5.shape,new_train_mask_5.shape,new_val_mask_5.shape,new_train_fnum_5.shape,new_val_fnum_5.shape))


log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_5.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model_5.fit([new_train_inp_5,new_train_mask_5],new_train_label_5,batch_size=8,epochs=1,validation_data=([new_val_inp_5,new_val_mask_5],new_val_label_5),callbacks=callbacks)

4504
4504
['8', '1', '8', '2', '8', '8', '8', '9', '7', '8', '1', '2', '1', '1', '8', '4', '8', '8', '12', '3', '3', '7', '3', '12', '1', '8', '8', '8', '8', '8', '8', '8', '1', '9', '5', '9', '9', '9', '11', '8', '8', '8', '4', '8', '8', '8', '8', '1', '3', '9', '3', '7', '1', '2', '9', '9', '7', '8', '8', '1', '10', '7', '8', '8', '9', '8', '7', '9', '9', '12', '7', '2', '8', '1', '11', '11', '1', '7', '7', '12', '1', '9', '8', '10', '12', '7', '8', '2', '8', '9', '9', '1', '8', '9', '1', '7', '12', '10', '10', '10', '8', '3', '7', '9', '8', '9', '1', '8', '8', '2', '7', '2', '9', '9', '11', '8', '8', '12', '12', '7', '8', '12', '4', '9', '3', '1', '12', '1', '1', '8', '8', '3', '8', '8', '8', '8', '9', '1', '8', '8', '10', '1', '8', '2', '8', '8', '7', '3', '8', '2', '4', '4', '9', '8', '10', '12', '12', '1', '1', '9', '1', '1', '1', '8', '2', '2', '8', '1', '1', '2', '2', '1', '2', '8', '1', '9', '9', '8', '8', '4', '2', '9', '9', '8', '3', '4', '3', '1', '8', '8', '2', '1', '9', '

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[ 8  1  2 ...  1 10  1]
[   1    2    4 ... 8414 8417 8406]
<class 'numpy.ndarray'>
7577
842
Train inp shape (4065, 512) Val input shape (439, 512)
Train label shape (4065,) Val label shape (439,)
Train attention mask shape (4065, 512) Val attention mask shape (439, 512)
Train filenumber shape (4065,) Val filenumber shape (439,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


509/509 [==============================] - 96s 173ms/step - loss: 7.4138 - accuracy: 0.5611 - val_loss: 6.6327 - val_accuracy: 0.6788


In [104]:
from collections import defaultdict
num_correct=0
def def_value():
    return 0

train_lab=[]

for i in range(len(new_val_fnum_0)):
  mod_labels=[]
  print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    #pred_test_0=model_0.predict([new_val_inp_0[i],new_val_mask_0[i]],batch_size=8)
    pred_test_0=model_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0 = pred_test_0.argmax(axis=1)
    mod_labels.append(pred_labels_0[0])

  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1=model_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1 = pred_test_1.argmax(axis=1)
    mod_labels.append(pred_labels_1[0])

  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2=model_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2 = pred_test_2.argmax(axis=1)
    mod_labels.append(pred_labels_2[0])

  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3=model_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3 = pred_test_3.argmax(axis=1)
    mod_labels.append(pred_labels_3[0])

  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4=model_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4 = pred_test_4.argmax(axis=1)
    mod_labels.append(pred_labels_4[0])

  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5=model_5.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5 = pred_test_5.argmax(axis=1)
    mod_labels.append(pred_labels_5[0])
  
  # mod_labels.append(pred_labels_1[i])
  # mod_labels.append(pred_labels_2[i])
  # mod_labels.append(pred_labels_3[i])
  # mod_labels.append(pred_labels_4[i])
  # mod_labels.append(pred_labels_5[i])

  res=max(set(mod_labels), key = mod_labels.count)
  # if(len(mod_labels)!=6):
  #   diff=6-len(mod_labels)
  #   for i in range(diff):
  #     mod_labels.append(-1)
  # train_lab.append(mod_labels)

  # print("Labels")
  # print(mod_labels)
  # print()
  # print("Predicted Label")
  # print(res)
  # print()
  # print("Actual Label")
  # print(new_val_label_0[i])
  # print('------------------')
  #res=pred_labels_5[i]
  if res==new_val_label_0[i]:
    num_correct=num_correct+1




#print(train_lab)
accuracy=num_correct/len(new_val_label_0)
print(accuracy)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27